In this notebook I am going to clean perform the data collection , data cleanning and data preprosessing and save the results in a csv file.

### Data collection

I will be collecting cleanned tweets from a local database , and save them in a pandas dataframe, and from that I will continue with my analysis.

In [15]:
from os import getenv
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
from urllib.parse import quote
from sqlalchemy.orm import sessionmaker
from pathlib import Path

def get_dot_env_file(ENV):
    if ENV:
        return Path.cwd().parent.joinpath(".env.dev")
    return find_dotenv(Path.cwd().parent.joinpath(".env.dev"))

def read_credentails(file='.env'):
    """
    Return users credentials from the environnement variable
    raise a an exception if the credentials are empty

    Raises:
        ValueError: raise a value error if no credentials was found
    """
    dot_env_file = get_dot_env_file(file)
    load_dotenv(dotenv_path=dot_env_file)
    DATABASE_HOST = getenv("POSTGRES_HOST")
    DATABASE_USER = getenv("POSTGRES_USER")
    DATABASE_PASSWORD = getenv("POSTGRES_PASSWORD")
    DATABASE_NAME = getenv("POSTGRES_DB")
    DATABASE_PORT = getenv('POSTGRES_PORT')
    all_variables = [DATABASE_HOST,
                     DATABASE_USER,
                     DATABASE_NAME,
                     DATABASE_PASSWORD,
                     DATABASE_PORT]
    if all(all_variables):
        return dict(zip(['database_host',
                         'database_user',
                         'database_name',
                         'database_password',
                         'database_port'], all_variables))
    else:
        raise ValueError(
            'Please add a .env file and put the credentials on it,\
                         refer to the sample')

In [16]:
def get_database_session(credentials):
    """
    Create a database session for database task

    Args:
        credentials (dict): credentials to use to connect to the db

    Returns:
        [tuple]: database session and the engine
    """
    database_url = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'\
        .format(user=credentials.get('database_user'),
                password=quote(credentials.get('database_password')),
                host=credentials.get('database_host'),
                database=credentials.get('database_name'),
                port=credentials.get('database_port'))
    engine = create_engine(database_url)
    Session = sessionmaker(bind=engine)
    session = Session()
    return session, engine

In [17]:
credentials = read_credentails('.env')
session, engine = get_database_session(credentials)

In [29]:
sql_query = "select text, created_at, raw_json from tweet"

In [26]:
import pandas as pd

In [30]:
data = None
with engine.connect() as connection:
    data = pd.read_sql_query(sql=sql_query, con=connection, parse_dates=['created_at'])

In [31]:
data.head()

,text,created_at,raw_json
0,size gdp economier member stater east african ...,2023-02-21 14:04:56,{'text': 'RT @TanzaniaInsight: The size of GDP...
1,execution convention apparent colonisation eco...,2023-02-21 14:04:49,"{'text': 'RT @StanysBujakera: #RDC: ""l'exécuti..."
2,,2023-02-21 14:04:42,"{'text': 'ハンドン様犬様と戯れてる', 'created_at': 1676988..."
3,today addi heads stater eac icglr reaffirmed s...,2023-02-21 14:04:40,{'text': 'RT @AmbMonicaJuma: 1/ Today in Addis...
4,,2023-02-21 14:04:33,{'text': '@sugiramireille @fernanddev @UNHuman...


In [32]:
data.shape

(2658300, 3)

In [33]:
data.to_parquet('data/tweets.parquet')

We can see in the table that we have all the tweets test and the time they where created, we those information we can go ahead and create our topic modelling modelt.

## Words representations

We all know that computer process information which is only in binary or number format , but in most of the nlp task we are dealing with text format.
This led to the question about the best way to store data for nlp task.? 
The first option that comes to our mind could be using one hot vector , each word can be encoded in a vector which has the size equals to the number of words in our vocabulary. This approach can work in example where we have a small vocabulary but for large vocabular it can lead to many issue since vector will grow larger and become sparse.
Fortunately we have others approaches we can uses , in this sections we will explore the most commons ones.

In this sections we will try differents words representation for our tweets, the most commons are: 
    - Bag of words 
    - Bag of n-grams : here we will try bigram and tri-grams
    - TF-IDF stands for term frequency times inverse document frequency.
    Term frequencies are the counts of each word in a document, which you
    learned about in previous chapters. Inverse document frequency means that
    you’ll divide each of those word counts
    - PMI pointwise mutual information

### bag of words

First let start with bag of words:

_what are bag of words_?

A bag word is a document representation were each words is represented with it frequency of occurence in a document or the number of times it occurs in a given document.

By document we denote a collection of words it can be a tweets, a blog post or a stackoverflow question.

Praticaly we use python counter to generate a dictionary of back of words with words and their frequency.

This representation has some drawbacks which are : 

- we new words and not comons words the lengths of our vectors will increase
- it can also lead to sparse vectors when we have rare words in a documents 



### Term frequency 

Wikipedia define the term frequency tf as a numerical statistics that intent to reflect how important a wort is in to document in a collection or a corpus.

$TF_{w, d} = \frac{n_{w,d}}{number of term in the document}$

In the numerator of the above formula we have the number of times a words appear in a document d and the denominator contains the number of words in a document.

### inverse document frequency 

The inverse document frequency is them measure of how important a words is in a corpus.

It is the log of the ratio between the total number of a documents in the corpus and the number of documents with the given word.

$Idf_t = \frac{nunber of documents }{nunber of document with the word w}$

This helps to penalise the most commons in a corpus those words carry less values for our analysis.

So the Term frequency inverse document frequency is the combinaison of both metrics , it helps to get how important a word is in a corpus and how impostant it is in a document

$(tf_idf)_{t,d } =  Idf_t * TF_{w, d}$

Pratically let get the inverse document frequency matrix for our corpus 

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
def vectorize_tweets(data, cleanned=True):
    '''
    this functions take the tweets data as parmeter 
    and return the vectorizeed version of the tweets
    '''
    tf_idf_vectorizer = TfidfVectorizer()
    if cleanned:
        data_tf_idf = tf_idf_vectorizer.fit_transform(data.cleanned_text)
        return data_tf_idf, tf_idf_vectorizer
    data_tf_idf = tf_idf_vectorizer.fit_transform(data.text)
    return data_tf_idf, tf_idf_vectorizer

we can see from this theat we have a large vocabulary with arround 215k words , this is huges and show that we may need some preprocessing in order to reduce the number of words in our corpus

### Improve word lematization

in this step I will be using lematization instead of stemming, 
Lemmatization is define as : 

_Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . If confronted with the token saw, stemming might return just s, whereas lemmatization would attempt to return either see or saw_ 

it will basically help to replace the words with their parent words in the base language :
    - the nouns nous `voulons` will mean `vouloir`

This approach has some advantages because it will help to not spread the information between different words forms derived from the same lemma, therefore it will lead to an accurate topic modelling because the same semantic information is assembled in one place.

It also have some disadvantages : 
new ambiguities may arise the words such as “je suis” should be rendered as “suivre” (to follow) or “être” (to be)

let now try to see it in pratices how it looks like

In [11]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [12]:
def lematize_text(text):
    doc = nlp(text)
    cleanned_text = " ".join([token.lemma_ for token in doc])
    return cleanned_text

In [13]:
data['cleanned_text'] = data.text.swifter.progress_bar(enable=True, desc='elapsed time when applying the function').apply(lematize_text)

In [17]:
data.head()

,id,text,created_at,cleanned_text
0,1260628656327725056,zambie militaire zambien occuper bout territoi...,2020-05-13 17:51:07,zambie militaire zambien occuper bout territoi...
1,1260628636710961153,covid essai clinique remède curatif bientôt dé...,2020-05-13 17:51:02,covid essai clinique remède curatif bientôt dé...
2,1260628592289030144,poursuivre justice .. val mieux contract gre g...,2020-05-13 17:50:52,poursuivre justice .. val mieux contract gre g...
3,1260628553504305153,jemapeleeeeeee madameeeee camp fcc bokola kooo...,2020-05-13 17:50:43,jemapeleeeeeee madameeee camp fcc bokola koooo...
4,1260628552426369024,fin poursuite judiciaire procès onem accueilli...,2020-05-13 17:50:42,fin poursuite judiciaire procès onem accueilli...


with new word lemmatization we can improve the pre processing step of those tweets and try to improve our model in productions<

In [18]:
data.to_csv('../data/cleanned_tweets_2021.csv')

let us try the tf idf vectorizer apain to examine how many vector we got 

https://dragonfly.hypotheses.org/648

https://stats.stackexchange.com/questions/179349/what-are-the-pros-and-cons-of-applying-pointwise-mutual-information-on-a-word-co


In [32]:
vectorized_data = vectorize_tweets(data, cleanned=False)

In [54]:
word_set = set()
for sentence in data.text:
    for word in sentence.split(' '):
        word_set.add(word)

In [55]:
cleanned_word_set = set()
for sentence in data.cleanned_text:
    for word in sentence.split(' '):
        cleanned_word_set.add(word)

In [58]:
len(word_set) - len(cleanned_word_set)

4098

with the cleanning process I gained somethilg arroud 4k words

#### Improve tokenization .

We can  also improve the preprocessing using a custom made tokenizer for social media data.

The NLTK library includes a tokenizer—casual_tokenize—that was built to deal
with short, informal, emoticon-laced texts from social networks where grammar and
spelling conventions vary widely.
The casual_tokenize function allows you to strip



source : the nltk book

This can be incorporate in the global text preprocessing pipeline

N gram before topic modeling 

In [78]:
vectorized_data, vectorizer = vectorize_tweets(data, cleanned=True)

We are still having a lot of word which are not written in the correct format and we may need to clean them using different cleanning techmics

- The first idea that comes to my mind is to use count vectorizer to count the most frequent word in the corpus and from that we can see if the misspelled word are frequent in the top words.

- I can also do some clustering using the leveinsten distance to see which word are similars to each other using the leveinsten distance and then group them in clusters

- I can also learn embedding from my word and check if the 2 version of the same words with almost the same spelling are similar using the cosine distance

- or I can learn the enmbedding but use cosine distance and the leveisten distance as similarity measure

with the cleanning we can see that the len of the word have reduced in ouw vocabulary

'100jour',
 '100jours',
 '100jpe',
 '100jr',
 '100jrs',

'28sept',
 '28septembre',
 '28settembr',

In [95]:
vectorized_data.shape

(1214425, 210907)

In [70]:
vectorized_data

<1214425x210907 sparse matrix of type '<class 'numpy.float64'>'
	with 8289054 stored elements in Compressed Sparse Row format>

With a comment I got  from a friend who is  NLP researchses  he said that that extra preprocessing step is not important for our project , we can start by building it from that and see how our result will behave and how the model perfom on differents evaluations metrics . From that we can come back on the cleanning process 